In [ ]:
%load_ext tensorboard

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from tqdm.notebook import tqdm

In [2]:
path="../Data/"

In [4]:
train=pd.read_csv(path+"Weusethis.csv")

In [ ]:
aaaa